# Importing the libraries

In [1]:
import numpy as np
import random
import os
import tensorflow as tf
from tensorflow import keras
import platform
from tensorflow.keras import layers, optimizers, losses, models, Input, Model
import time # Per misurare il tempo di training
from tensorflow.keras.callbacks import EarlyStopping # Per l'early stopping
import matplotlib.pyplot as plt
from tqdm import tqdm # Per mostrare una barra di progresso

# Set seeds for random operations.

In [2]:
# --- 1. Impostazione del Seed Globale all'inizio del tuo script ---
# Questo è il punto chiave per la riproducibilità di TUTTO ciò che segue.
MASTER_RANDOM_SEED = 42
np.random.seed(MASTER_RANDOM_SEED)
random.seed(MASTER_RANDOM_SEED) # Imposta anche il seed per la libreria 'random' di Python se la usi
tf.random.set_seed(MASTER_RANDOM_SEED)
os.environ['PYTHONHASHSEED'] = str(MASTER_RANDOM_SEED) # Per operazioni basate su hash (es. ordine dei dizionari)
os.environ['TF_DETERMINISTIC_OPS'] = '1' # Forza operazioni deterministiche in TensorFlow 2.x

# Print the HW Specs.

In [3]:
print("--- Dettagli dell'Architettura Hardware della Sessione Colab ---\n")

# --- 1. Dettagli CPU ---
print("--- Dettagli CPU ---")
!lscpu
print("\n")

# --- 2. Dettagli RAM (Memoria) ---
print("--- Dettagli RAM (Memoria) ---")
!cat /proc/meminfo | grep MemTotal
print("\n")

# --- 3. Dettagli Spazio su Disco ---
print("--- Dettagli Spazio su Disco ---")
!df -h /
print("\n")

# --- 4. Dettagli Acceleratore Hardware (GPU/TPU) ---
print("--- Dettagli Acceleratore Hardware (GPU/TPU) ---")
try:
    tpu_address = os.environ.get('COLAB_TPU_ADDR')
    if tpu_address:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=tpu_address)
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        print(f"Tipo Acceleratore: TPU (indirizzo: {tpu_address})")
        print("Dispositivi TPU disponibili:")
        for device in tf.config.list_logical_devices('TPU'):
            print(f"  - {device.name}")
    else:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            print(f"Tipo Acceleratore: GPU")
            for gpu in gpus:
                print(f"  - Dispositivo GPU rilevato: {gpu.name}")
            print("\nDettagli GPU specifici (da `!nvidia-smi`):")
            !nvidia-smi
        else:
            print("Tipo Acceleratore: Nessuna GPU o TPU rilevata (in uso CPU)")

except Exception as e:
    print(f"Si è verificato un errore durante la rilevazione dell'acceleratore: {e}")
    print("Tentativo di rilevare i dispositivi TensorFlow standard:")
    devices = tf.config.list_logical_devices()
    if devices:
        for device in devices:
            print(f"  - Dispositivo rilevato: {device.name}, Tipo: {device.device_type}")
    else:
        print("Nessun dispositivo TensorFlow rilevato.")

print("\n--- Analisi Dettagli Hardware Completata ---")

--- Dettagli dell'Architettura Hardware della Sessione Colab ---

--- Dettagli CPU ---
Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.00GHz
    CPU family:           6
    Model:                85
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             3
    BogoMIPS:             4000.37
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht sysc
                          all nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xt
                          opology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq
                           ssse3 fma cx16 pcid sse4_1 sse4

# Connect To Gdrive to store the datasets created.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Define the paths

In [5]:
# Percorsi dataset
paths = {
    "0-20": {
        "train": "/content/drive/MyDrive/GitHub/AWGN/dataset/training_0-20_SNR.npz",
        "val": "/content/drive/MyDrive/GitHub/AWGN/dataset/validation_0-20_SNR.npz",
    },
    "11-15": {
        "train": "/content/drive/MyDrive/GitHub/AWGN/dataset/training_11-15_SNR.npz",
        "val": "/content/drive/MyDrive/GitHub/AWGN/dataset/validation_11-15_SNR.npz",
    }
}

# Directory salvataggio modelli
save_dir = "/content/drive/MyDrive/GitHub/AWGN/trained_models/cnn"
os.makedirs(save_dir, exist_ok=True)

# Load the datasets.

In [6]:
def load_dataset(filepath):
    """
    Carica i dati X e y da un file .npz.
    Si aspetta che il file contenga 'X_train' o 'X_val' e 'y_train' o 'y_val'.
    """
    data = np.load(filepath)

    # Controlla se le chiavi per il training sono presenti
    if 'X_train' in data and 'y_train' in data:
        print(f"  Caricato Training data da: {filepath}")
        return data['X_train'], data['y_train']
    # Altrimenti, controlla se le chiavi per la validation sono presenti
    elif 'X_val' in data and 'y_val' in data:
        print(f"  Caricato Validation data da: {filepath}")
        return data['X_val'], data['y_val']
    else:
        # Se nessuna delle combinazioni attese è trovata, solleva un errore
        raise ValueError(f"Il file {filepath} non contiene i dati X e y attesi (né 'X_train'/'y_train' né 'X_val'/'y_val'). "
                         f"Chiavi trovate: {list(data.keys())}")

# CNN model definition

In [7]:
# Definizione CNN con filtri e kernel size variabili
def build_cnn(input_shape, num_filters, kernel_size):
    model_id = f"CNN_{kernel_size}-ksize_{num_filters}-filters"
    inputs = Input(shape=input_shape)
    x = layers.Reshape((input_shape[0], 1))(inputs)
    x = layers.Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu', padding='same')(x)
    x = layers.Dropout(0.4)(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(2, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs, name=model_id)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model, model_id

# Training...

In [11]:
# Parametri da testare
filter_options = [128, 256, 512, 1024]
kernel_sizes = [3, 5, 7, 9]

# Addestramento su entrambi i range SNR
for snr_range, datasets in paths.items():
    x_train, y_train = load_dataset(datasets['train'])
    x_val, y_val = load_dataset(datasets['val'])

    for filters in filter_options:
        for ksize in kernel_sizes:
            model_id = f"CNN_{ksize}-ksize_{filters}-filters"
            model_filename = f"{model_id}_awgn_snr_{snr_range}.h5"
            model_path = os.path.join(save_dir, model_filename)

            if os.path.exists(model_path):
                print(f"[SKIP] Modello già esistente: {model_filename}")
                continue

            print(f"\n>>> Training CNN con {filters} filtri, kernel size {ksize}, SNR {snr_range}")
            model, _ = build_cnn(x_train.shape[1:], filters, ksize)

            es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
            model.fit(
                x_train, y_train,
                validation_data=(x_val, y_val),
                epochs=30,
                batch_size=32,
                callbacks=[es],
                verbose=1
            )

            model.save(model_path)
            print(f"[SALVATO] Modello: {model_filename}")

  Caricato Training data da: /content/drive/MyDrive/GitHub/AWGN/dataset/training_0-20_SNR.npz
  Caricato Validation data da: /content/drive/MyDrive/GitHub/AWGN/dataset/validation_0-20_SNR.npz
[SKIP] Modello già esistente: CNN_3-ksize_128-filters_awgn_snr_0-20.h5
[SKIP] Modello già esistente: CNN_5-ksize_128-filters_awgn_snr_0-20.h5
[SKIP] Modello già esistente: CNN_7-ksize_128-filters_awgn_snr_0-20.h5
[SKIP] Modello già esistente: CNN_9-ksize_128-filters_awgn_snr_0-20.h5
[SKIP] Modello già esistente: CNN_3-ksize_256-filters_awgn_snr_0-20.h5
[SKIP] Modello già esistente: CNN_5-ksize_256-filters_awgn_snr_0-20.h5
[SKIP] Modello già esistente: CNN_7-ksize_256-filters_awgn_snr_0-20.h5
[SKIP] Modello già esistente: CNN_9-ksize_256-filters_awgn_snr_0-20.h5
[SKIP] Modello già esistente: CNN_3-ksize_512-filters_awgn_snr_0-20.h5
[SKIP] Modello già esistente: CNN_5-ksize_512-filters_awgn_snr_0-20.h5
[SKIP] Modello già esistente: CNN_7-ksize_512-filters_awgn_snr_0-20.h5
[SKIP] Modello già esistent

[SALVATO] Modello: CNN_7-ksize_512-filters_awgn_snr_11-15.h5

>>> Training CNN con 512 filtri, kernel size 9, SNR 11-15
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 14ms/step - accuracy: 0.9916 - loss: 0.0180 - val_accuracy: 1.0000 - val_loss: 8.7221e-07
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 25s 13ms/step - accuracy: 1.0000 - loss: 1.2797e-06 - val_accuracy: 1.0000 - val_loss: 7.5586e-08
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 14ms/step - accuracy: 1.0000 - loss: 2.7016e-08 - val_accuracy: 1.0000 - val_loss: 6.3439e-08
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 26s 14ms/step - accuracy: 1.0000 - loss: 3.1656e-08 - val_accuracy: 1.0000 - val_loss: 1.2683e-08
Epoch 5/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 25s 13ms/step - accuracy: 1.0000 - loss: 2.6013e-09 - val_accuracy: 1.0000 - val_loss: 9.4944e-09
Epoch 6/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 14ms/step - accuracy: 0.9987 - loss: 0.0057 - val_accuracy: 1.0000 - val_loss: 2.3923e-05
Epoch 7/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 40s 13

[SALVATO] Modello: CNN_9-ksize_512-filters_awgn_snr_11-15.h5

>>> Training CNN con 1024 filtri, kernel size 3, SNR 11-15
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 44s 23ms/step - accuracy: 0.9861 - loss: 0.0317 - val_accuracy: 1.0000 - val_loss: 8.1727e-05
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 1.0000 - loss: 1.4044e-06 - val_accuracy: 1.0000 - val_loss: 6.5604e-05
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 22ms/step - accuracy: 0.9996 - loss: 0.0025 - val_accuracy: 0.9997 - val_loss: 0.0010
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 1.0000 - loss: 9.0850e-06 - val_accuracy: 0.9994 - val_loss: 0.0027
Epoch 5/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.9975 - loss: 0.0117 - val_accuracy: 0.9998 - val_loss: 7.6246e-04
Epoch 6/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 1.0000 - loss: 8.2018e-07 - val_accuracy: 1.0000 - val_loss: 5.4253e-08
Epoch 7/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 22ms/step - a

[SALVATO] Modello: CNN_3-ksize_1024-filters_awgn_snr_11-15.h5

>>> Training CNN con 1024 filtri, kernel size 5, SNR 11-15
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 43s 22ms/step - accuracy: 0.9887 - loss: 0.0222 - val_accuracy: 1.0000 - val_loss: 7.5097e-08
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 22ms/step - accuracy: 1.0000 - loss: 5.0590e-08 - val_accuracy: 1.0000 - val_loss: 2.8804e-08
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 81s 22ms/step - accuracy: 1.0000 - loss: 1.7444e-08 - val_accuracy: 1.0000 - val_loss: 1.5419e-08
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 22ms/step - accuracy: 1.0000 - loss: 1.4645e-08 - val_accuracy: 1.0000 - val_loss: 1.4305e-09
Epoch 5/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 22ms/step - accuracy: 1.0000 - loss: 3.1355e-09 - val_accuracy: 1.0000 - val_loss: 1.0729e-09
Epoch 6/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.9990 - loss: 0.0071 - val_accuracy: 1.0000 - val_loss: 1.7613e-05
Epoch 7/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 

[SALVATO] Modello: CNN_5-ksize_1024-filters_awgn_snr_11-15.h5

>>> Training CNN con 1024 filtri, kernel size 7, SNR 11-15
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 43s 22ms/step - accuracy: 0.9891 - loss: 0.0244 - val_accuracy: 1.0000 - val_loss: 1.6590e-06
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 22ms/step - accuracy: 0.9956 - loss: 0.0233 - val_accuracy: 1.0000 - val_loss: 1.1471e-05
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 81s 22ms/step - accuracy: 1.0000 - loss: 2.7819e-05 - val_accuracy: 1.0000 - val_loss: 2.1212e-06
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 22ms/step - accuracy: 1.0000 - loss: 4.1271e-05 - val_accuracy: 1.0000 - val_loss: 4.0124e-06
Epoch 5/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 22ms/step - accuracy: 1.0000 - loss: 1.9598e-06 - val_accuracy: 1.0000 - val_loss: 1.0216e-06
Epoch 6/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 22ms/step - accuracy: 1.0000 - loss: 3.0841e-07 - val_accuracy: 1.0000 - val_loss: 2.7914e-07
Epoch 7/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 81s 

[SALVATO] Modello: CNN_7-ksize_1024-filters_awgn_snr_11-15.h5

>>> Training CNN con 1024 filtri, kernel size 9, SNR 11-15
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 44s 23ms/step - accuracy: 0.9876 - loss: 0.0246 - val_accuracy: 1.0000 - val_loss: 9.2662e-07
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 80s 22ms/step - accuracy: 1.0000 - loss: 2.2686e-07 - val_accuracy: 1.0000 - val_loss: 9.2729e-08
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 22ms/step - accuracy: 1.0000 - loss: 2.0670e-08 - val_accuracy: 1.0000 - val_loss: 6.6155e-08
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 22ms/step - accuracy: 1.0000 - loss: 2.7061e-08 - val_accuracy: 1.0000 - val_loss: 4.6371e-09
Epoch 5/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 81s 22ms/step - accuracy: 1.0000 - loss: 5.7409e-09 - val_accuracy: 1.0000 - val_loss: 1.0996e-08
Epoch 6/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 83s 22ms/step - accuracy: 1.0000 - loss: 1.4937e-10 - val_accuracy: 1.0000 - val_loss: 3.2007e-09
Epoch 7/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 

[SALVATO] Modello: CNN_9-ksize_1024-filters_awgn_snr_11-15.h5
